In [ ]:
import rpy2
print(rpy2.__version__)

In [ ]:
from rpy2.robjects.packages import importr
# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

packnames = ('ggplot2', 'spam64')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

In [ ]:
# Code for writing out surface ply file with colormap created by matplotlib, e.g., to be imported into Blender

def write_plyRGB(filename, vertices, faces, colorsR,colorsG,colorsB,comment=None):
    import pandas as pd
    print("writing ply format")
    # infer number of vertices and faces
    number_vertices = vertices.shape[0]
    number_faces = faces.shape[0]
    # make header dataframe
    header = ['ply',
            'format ascii 1.0',
            'comment %s' % comment,
            'element vertex %i' % number_vertices,
            'property float x',
            'property float y',
            'property float z',
            'property uchar red',
            'property uchar green',
            'property uchar blue',
            'element face %i' % number_faces,
            'property list uchar int vertex_indices',
            'end_header'
             ]
    header_df = pd.DataFrame(header)
    # make dataframe from vertices
    vertex_df = pd.DataFrame(vertices/50)
    #colors_df = pd.DataFrame(np.tile(np.round(colors/7*255), (3,1)).T)
    ColorsR_df=pd.DataFrame(colorsR)
    ColorsG_df=pd.DataFrame(colorsG)
    ColorsB_df=pd.DataFrame(colorsB)
    colorsConcat = pd.concat([ColorsR_df,ColorsG_df,ColorsB_df], axis=1)
    colors_df=pd.DataFrame(colorsConcat)
    colors_df=colorsConcat.astype(int)
    df_concat = pd.concat([vertex_df, colors_df], axis=1)
    # make dataframe from faces, adding first row of 3s (indicating triangles)
    triangles = np.reshape(3 * (np.ones(number_faces)), (number_faces, 1))
    triangles = triangles.astype(int)
    faces = faces.astype(int)
    faces_df = pd.DataFrame(np.concatenate((triangles, faces), axis=1))
    # write dfs to csv
    header_df.to_csv(filename, header=None, index=False)
    with open(filename, 'a') as f:
        df_concat.to_csv(f, header=False, index=False,
                         float_format='%.3f', sep=' ')
    with open(filename, 'a') as f:
        faces_df.to_csv(f, header=False, index=False,
                        float_format='%.0f', sep=' ')

In [ ]:
# Code to read cifti data into python. 

import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt


def surf_data_from_cifti(data, axis, surf_name): 
    assert isinstance(axis, nb.cifti2.BrainModelAxis)
    for name, data_indices, model in axis.iter_structures():  # Iterates over volumetric and surface structures
        if name == surf_name:                                 # Just looking for a surface
            data = data.T[data_indices]                       # Assume brainmodels axis is last, move it to front
            vtx_indices = model.vertex                        # Generally 1-N, except medial wall vertices
            surf_data = np.zeros((vtx_indices.max() + 1,) + data.shape[1:], dtype=data.dtype)
            surf_data[vtx_indices] = data
            return surf_data
    raise ValueError(f"No structure named {surf_name}")


In [ ]:
# Load surface Midnight Scan Club derivates data downloaded from openneuro website

import glob

left_brain=np.zeros([32492,10,3])

# Loop through the file list and read each file
dir_path='MSCderivatives/surface_pipeline/'
for sub in range(10):

    file_pattern = 'sub-MSC' + str(sub+1).zfill(2) + '/task_contrasts_cifti/*/' + '*fsLR_smooth2.55.dscalar.nii'
    #file_pattern = 'sub-MSC' + str(sub+1).zfill(2) + '/task_contrasts_cifti/*/' + '*fsLR.dscalar.nii'
    
    file_list = glob.glob(f"{dir_path}/{file_pattern}")

    
    for i, fileName in enumerate(file_list):
        print(fileName)
        cifti = nb.load(fileName)
        cifti_data = cifti.get_fdata(dtype=np.float32)
        cifti_hdr = cifti.header
        nifti_hdr = cifti.nifti_header
        axes = [cifti_hdr.get_axis(i) for i in range(cifti.ndim)]
        
        
        left_brain[:,sub,i]=surf_data_from_cifti(cifti_data, axes[1], 'CIFTI_STRUCTURE_CORTEX_LEFT')[:,0]*-1 # Remove -1 to look at predict task negative
        


In [ ]:
# Sanity check of projection onto surface and load 3D coordinates and faces of FSLR32k MSM surface for visualisation
# Surface downloaded from HCP group data

import matplotlib.pyplot as plt

gifti_img_Midthickness = nb.load('S900.L.midthickness_MSMAll.32k_fs_LR.surf.gii')
xyz_points_Mid=gifti_img_Midthickness.darrays[0].data
azim=180
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
print(left_brain.shape)

p=ax.scatter(xyz_points_Mid[:,0],xyz_points_Mid[:,1],xyz_points_Mid[:,2],c=np.where(left_brain[:,1,1]>1,left_brain[:,1,1],0),s=5,cmap='bwr_r',vmin=-2,vmax=2)
ax.view_init(elev=10., azim=azim)
fig.colorbar(p,ax=ax)


In [ ]:
# convert from R3 (euclidean xyz) to S2 (spherical) coordinates and load FSLR32k spherical projection coordinates
# Surface downloaded from HCP group data

import nibabel as nb
import numpy as np
 
gifti_img_BaseBrain = nb.load('S900.L.sphere.32k_fs_LR.surf.gii')

xyz_points=gifti_img_BaseBrain.darrays[0].data


def appendSpherical_np(xyz):
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    
    xy = xyz[:,0]**2 + xyz[:,1]**2
    ptsnew[:,3] = np.sqrt(xy + xyz[:,2]**2)
    ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,0])
    return ptsnew

ptsnew=appendSpherical_np(xyz_points)


In [ ]:
# Run lattice kriging on lowest 25% of vertices

import numpy as np
from rpy2.robjects.packages import importr
from rpy2.robjects import FloatVector
from rpy2.robjects import r, pandas2ri
import pandas as pd
import scipy
from sklearn.neighbors import DistanceMetric

pandas2ri.activate()
LK = importr('LatticeKrig')


LKPred=np.zeros(left_brain.shape)

for i in range(left_brain.shape[1]):
    for j in range(left_brain.shape[2]):

        NotZero=np.abs(left_brain[:,i,j])>0
        threshUpper= np.percentile(left_brain[NotZero,i,j], 75, axis=0)
        threshLower= np.percentile(left_brain[NotZero,i,j], 25, axis=0)
        


        print(threshUpper)

        pos_index=left_brain[:,i,j]>threshUpper
        #neg_index=left_brain[:,task]<-3

        X_temp=np.degrees(ptsnew[pos_index,4])

        X1=FloatVector(X_temp[:])

        X_temp=np.degrees(ptsnew[pos_index,5])
        X2=FloatVector(X_temp[:])
        Y_temp=left_brain[pos_index,i,j]
        Y=FloatVector(Y_temp[:])


        df = pd.DataFrame({'X1': X2, 'X2': X1})

        r_dataframe = pandas2ri.py2rpy(df)


        alpha=FloatVector(np.power([1,0.5,0.1],2))

        LKinfo=LK.LKrigSetup(r_dataframe,startingLevel=3,nlevel=3,a_wght=1.01,alpha=alpha,LKGeometry="LKSphere",Radius=100) 

        LKOutput=LK.LatticeKrig(r_dataframe,Y,LKinfo=LKinfo)

        X1_all=FloatVector(np.degrees(ptsnew[:,4]))
        X2_all=FloatVector(np.degrees(ptsnew[:,5]))
        Y_all=FloatVector(left_brain[:,i,j])


        df_all = pd.DataFrame({'X1': X2_all, 'X2': X1_all})

        r_dataframe_all = pandas2ri.py2rpy(df_all)
        LKPred[:,i,j]=r.predict(LKOutput,r_dataframe_all).ravel()

        
        





In [ ]:
# Write out .ply surfaces of interpolated data for each task for each participant.
import  matplotlib.cm
cmap = matplotlib.cm.get_cmap('coolwarm_r')
vertices=gifti_img_Midthickness.darrays[0].data
faces=gifti_img_Midthickness.darrays[1].data
for task in range(3):
    for sub in range(10):
    
        NotZero=np.abs(left_brain[:,sub,task])>0
        threshUpper= np.percentile(left_brain[NotZero,sub,task], 75, axis=0)

        colors=left_brain[:,sub,task]
        norm = matplotlib.colors.Normalize(vmin=-2, vmax=2)
        CMap=cmap(norm(colors))*255
        PlyFileName='MSCActivitySmoothed' + str(task) +'_sub_' + str(sub) + '.ply'
        write_plyRGB(PlyFileName,vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])

        colors=np.zeros(left_brain[:, sub,task].shape)
        colors[left_brain[:,sub,task]>threshUpper]=left_brain[left_brain[:,sub,task]>threshUpper,sub,task]
        norm = matplotlib.colors.Normalize(vmin=-2, vmax=2)
        CMap=cmap(norm(colors))*255
        PlyFileName='MSCThreshSmoothed' + str(task) +'_sub_' + str(sub) + '.ply'
        write_plyRGB(PlyFileName,vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])

        colors=np.zeros(left_brain[:, sub,task].shape)
        colors[NotZero]=LKPred[NotZero,sub,task]
        NotZero
        norm = matplotlib.colors.Normalize(vmin=-2, vmax=2)
        CMap=cmap(norm(colors))*255
        PlyFileName='MSCprojectedSmoothed' + str(task) +'_sub_' + str(sub) + '.ply'
        write_plyRGB(PlyFileName,vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])



In [ ]:
# Write out .ply files for real and interpolated maps averaged over the group.
for task in range(3):
    
    colors=left_brain.mean(axis=1)[:,task]
    norm = matplotlib.colors.Normalize(vmin=-2, vmax=2)
    CMap=cmap(norm(colors))*255
    PlyFileName='MSCSmoothGroupAvActivity' + str(task) + '.ply'
    write_plyRGB(PlyFileName,vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])

    colors=np.zeros(left_brain[:, 0,task].shape)
    colors[NotZero]=LKPred.mean(axis=1)[NotZero,task]
    NotZero
    norm = matplotlib.colors.Normalize(vmin=-2, vmax=2)
    CMap=cmap(norm(colors))*255
    PlyFileName='MSCSmoothGroupAvProjected' + str(task) + '.ply'
    write_plyRGB(PlyFileName,vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])






In [ ]:
# Visualise some real and predicted within jupyter notebook as a sanity check in jupyter notebook 
import matplotlib.pyplot as plt


gifti_img_Midthickness = nb.load('/Users/robleech/Dropbox/HCP_S900_GroupAvg_v1/S900.L.midthickness_MSMAll.32k_fs_LR.surf.gii')
xyz_points_Mid=gifti_img_Midthickness.darrays[0].data
azim=180

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

p=ax.scatter(xyz_points_Mid[:,0],xyz_points_Mid[:,1],xyz_points_Mid[:,2],c=left_brain[:,3,2],s=3,cmap='bwr_r',vmin=-5,vmax=5)
ax.view_init(elev=10., azim=azim)
fig.colorbar(p,ax=ax)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

p=ax.scatter(xyz_points_Mid[:,0],xyz_points_Mid[:,1],xyz_points_Mid[:,2],c=LKPred[:,3,2],s=3,cmap='bwr_r',vmin=-5,vmax=5)
ax.view_init(elev=10., azim=azim)
fig.colorbar(p,ax=ax)


In [ ]:
# With latticeKrig predict task activity in conjunction mask from each pair of tasks.

from matplotlib import pyplot as plt
import numpy as np
from rpy2.robjects.packages import importr
from rpy2.robjects import FloatVector
from rpy2.robjects import r, pandas2ri
import pandas as pd
import scipy
from sklearn.neighbors import DistanceMetric
from scipy import stats


pandas2ri.activate()

tasks=np.asarray([0,1,2])

LKPredAllTasks=np.zeros([10,tasks.shape[0],tasks.shape[0]])
LKPredAllTasksAlt=np.zeros([10,tasks.shape[0],tasks.shape[0]])
LKPredAllTasksTNRestrict=np.zeros([10,tasks.shape[0],tasks.shape[0]])
LKPredAllTasksTNRestrictAlt=np.zeros([10,tasks.shape[0],tasks.shape[0]])

LK = importr('LatticeKrig')
for i in range(10):
    print('subject' + str(i))
    for u in range(tasks.shape[0]):
        print(u)
        for v in range(tasks.shape[0]):
            print(v)
            #print(v)
            #if u !=v: 
            tasksCompare=np.array([u,v])
            NotZero1=np.abs(left_brain[:,i,u])>0
            NotZero2=np.abs(left_brain[:,i,v])>0
            threshTask1= np.percentile(left_brain[NotZero1,i,u], 75, axis=0)
            threshTask2= np.percentile(left_brain[NotZero2,i,v], 75, axis=0)

            pos_index=np.logical_and(left_brain[:,i,u]>threshTask1,left_brain[:,i,v]>threshTask2)
            neg_index=np.logical_not(left_brain[:,i,u]>threshTask1,left_brain[:,i,v]>threshTask2)
            #neg_index=np.logical_and(left_brain[:,tasksCompare[0]]<0,left_brain[:,tasksCompare[1]]<0)

            #pos_index=np.logical_and(left_brain[:,tasksCompare[0]]>10,left_brain[:,tasksCompare[1]]>10)
            #neg_index=np.logical_not(left_brain[:,tasksCompare[0]]>10,left_brain[:,tasksCompare[1]]>10)

            task=u
            
            X_temp=np.degrees(ptsnew[pos_index,4])

            X1=FloatVector(X_temp[:])

            X_temp=np.degrees(ptsnew[pos_index,5])
            X2=FloatVector(X_temp[:])
            Y_temp=left_brain[pos_index,i,task]
            Y=FloatVector(Y_temp[:])

            df = pd.DataFrame({'X1': X2, 'X2': X1})

            r_dataframe = pandas2ri.py2rpy(df)

            alpha=FloatVector(np.power([1,0.5,0.1],2))

            LKinfo=LK.LKrigSetup(r_dataframe,startingLevel=3,nlevel=3,a_wght=1.01,alpha=alpha,LKGeometry="LKSphere",Radius=100) 

            LKOutput=LK.LatticeKrig(r_dataframe,Y,LKinfo=LKinfo)

            X1_all=FloatVector(np.degrees(ptsnew[:,4]))
            X2_all=FloatVector(np.degrees(ptsnew[:,5]))
            Y_all=FloatVector(left_brain[:,i,task])


            df_all = pd.DataFrame({'X1': X2_all, 'X2': X1_all})

            r_dataframe_all = pandas2ri.py2rpy(df_all)
            tempPredict=r.predict(LKOutput,r_dataframe_all).ravel()

            LKPredAllTasks[i,u,v]=stats.spearmanr(tempPredict[neg_index],left_brain[neg_index,i,u])[0]
            LKPredAllTasksAlt[i,u,v]=stats.spearmanr(tempPredict[neg_index],left_brain[neg_index,i,v])[0]
            LKPredAllTasksTNRestrict[i,u,v]=stats.spearmanr(tempPredict[left_brain[:,i,u]<0],left_brain[left_brain[:,i,u]<0,i,u])[0]
            LKPredAllTasksTNRestrictAlt[i,u,v]=stats.spearmanr(tempPredict[left_brain[:,i,u]<0],left_brain[left_brain[:,i,u]<0,i,v])[0]


In [ ]:
# Display across task similarity for group
print(LKPredAllTasks.shape)
fig = plt.figure(figsize=(8, 8))
plt.pcolormesh((LKPredAllTasks.mean(axis=0)-LKPredAllTasksAlt.mean(axis=0)), edgecolors='k', linestyle='-',linewidth=1,vmin=-0.5,vmax=0.5,cmap='bwr')
plt.xticks([])
plt.yticks([])
plt.savefig("MSCIntersectionPermuteSmooth.png", bbox_inches='tight')
plt.show()


In [ ]:
# Display across task similarity for individuals

for i in range(10):
    #plt.imshow(LKPredAllTasks[i,:,:]-LKPredAllTasksAlt[i,:,:],cmap='bwr',vmin=-1,vmax=1)
   
    fig = plt.figure(figsize=(4, 4))
    #plt.imshow(LKPredAllTasks-LKPredAllTasksAlt,vmin=-0.5,vmax=0.5,cmap='bwr')
    plt.pcolormesh(LKPredAllTasks[i,:,:]-LKPredAllTasksAlt[i,:,:], edgecolors='k', linestyle='-',linewidth=1,vmin=-0.5,vmax=0.5,cmap='bwr')
    #plt.pcolormesh(LKPredAllTasksTNRestrict[i,:,:]-LKPredAllTasksTNRestrictAlt[i,:,:], edgecolors='k', linestyle='-',linewidth=1,vmin=-0.5,vmax=0.5,cmap='bwr')
    
    plt.xticks([])
    plt.yticks([])
    plt.savefig('MSCSmoothIntersectionPermute_Sub' + str(i) + '.png', bbox_inches='tight')
    #plt.imshow(LKPredAllTasksTNRestrict-LKPredAllTasksTNRestrictAlt,vmin=-0.5,vmax=0.5,cmap='bwr')
    #plt.imshow(LKPredAllTasksMinDist-LKPredAllTasksAltMinDist,vmin=-0.5,vmax=0.5,cmap='bwr')
    #plt.colorbar()
    plt.show()

